In [1]:
import re, os
import requests
from urllib.parse import urlparse
from tqdm import tqdm
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import time


In [2]:
os.chdir(r"C:\Users\daV\Documents\ZHAW\HS 2024\dPoDL\dPoDL\experiments\github-downloads")
csvs = r"C:\Users\daV\Documents\ZHAW\HS 2024\dPoDL\dPoDL\experiments\github-csvs"

In [3]:
# Define individual regex patterns for each metric for now just focus on relaxation of problem; LOSS
epoch_loss_pattern = re.compile(r'epoch (\d+)/\d+.*? loss:\s*(\d+\.\d+)', re.DOTALL)
#epoch_loss_val_loss_pattern1 = re.compile(r'epoch (\d+)/\d+.*? loss:\s*(\d+\.\d+).*? val_loss:\s*(\d+\.\d+)' , re.DOTALL)
#epoch_loss_val_loss_pattern2 = re.compile(r'epoch (\d+)/\d+.*? val_loss:\s*(\d+\.\d+).*? loss:\s*(\d+\.\d+)', re.DOTALL)

c = 0

def regex_metrics_tf(text_original):
    text_original = text_original.lower()
    dfs = []
    
    # split everytime new epoch count is detected -> multiple trainings in one notebook
    keyword = "epoch 1/"
    text_splits = re.split(f'(?<={keyword})', text_original)[1:]    # Use regex with lookbehind to keep the keyword in the result
    text_splits = ["epoch 1/" + s  for s in text_splits]
    
    for ix, text in enumerate(text_splits):
        #start = time.time()
        epoch_loss_matches = epoch_loss_pattern.findall(text)
        #print(time.time()-start, "first search")
        
        if len(epoch_loss_matches) > 0:
            df = pd.DataFrame(epoch_loss_matches, columns=["Epoch", "Loss"])
        else:
            df = pd.DataFrame({})
        dfs.append(df)
        
    return dfs


def wrapper(filename):
    file_out = os.path.splitext(filename)[0] + f"-1.csv"
    file_out = os.path.join(csvs, file_out)
    if os.path.isfile(file_out): 
        return
    file = open(filename, "r").read()        
    a = regex_metrics_tf(file)
    for ix, df in enumerate(a):
        if df.empty:
            c += 1
            #print(filename)
            break
        else:
            file_out = file_out.replace("-1.csv", f"-{ix+1}.csv")
            df.to_csv(file_out, index=False)
    return a


#test_item = os.listdir()[2322]  #2322   sample 10  -> 82s
#a = wrapper(test_item)


with ThreadPoolExecutor() as executor:
    executor.map(wrapper, os.listdir()) 


In [4]:
c

0